In [130]:
import time
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import requests
import matplotlib.pyplot as plt
import seaborn as sns

### Спарсим данные, сохраним полученные датафреймы и будем в случае чего обращаться к локально сохраненным данным

In [128]:
# Парсер для получения данных с westmetall
def parser(metall_str: str):
    url = f'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall_str}_cash'
        
    response = requests.get(url=url)
    
    # преобразуем ответ в датафрейм
    df_2023 = pd.read_html(response.text)[0]
    df_2022 = pd.read_html(response.text)[1]
    df_2021 = pd.read_html(response.text)[2]
    df_2020 = pd.read_html(response.text)[3]
    
    df = pd.concat([
        df_2023, df_2022, df_2021, df_2020
    ]
    )
    
    # очищаем таблицу от неинформативных строк
    cleared_df = df[df['date'] != 'date']
    # заменяем символы пропусков нулями
    cleared_df = cleared_df.replace('-', 0)
    
    # приобразуем типы данных
    cleared_df['date'] = pd.to_datetime(cleared_df['date'])
    # т.к. столбцы имеют разное назание, обращяемся по индексу
    cleared_df.iloc[:,1] = pd.to_numeric(
        cleared_df.iloc[:,1])

    cleared_df.iloc[:,2] = pd.to_numeric(
        cleared_df.iloc[:,2])
    # в данном столбце, за счет использования символа запятой
    # для разделения целой части от дробной,
    # pandas не корректно преобразовал значения
    cleared_df.iloc[:,3] = pd.to_numeric(
        cleared_df.iloc[:,3]) / 1000
    
    return cleared_df.set_index('date')

# Список источников, нам понадобится информация по 5 видам сырья:
# алюминий, медь, свинец, никель и цинк
metalls = ['Al', 'Cu', 'Pb', 'Ni', 'Zn']

# Иниуиируем датафреймы, делая паузу после исполнения, для обхода
# возможной блокировки со стороны источника
al_df = parser(metalls[0])
time.sleep(1)
cu_df = parser(metalls[1])
time.sleep(1)
pb_df = parser(metalls[2])
time.sleep(1)
ni_df = parser(metalls[3])
time.sleep(1)
zn_df = parser(metalls[4])

In [129]:
# Выведем полученные данные и сохраним как бекап.
al_df.to_csv('./data/aluminium.csv', sep=',', index=False)
cu_df.to_csv('./data/copper.csv', sep=',', index=False)
pb_df.to_csv('./data/lead.csv', sep=',', index=False)
ni_df.to_csv('./data/nickel.csv', sep=',', index=False)
zn_df.to_csv('./data/zink.csv', sep=',', index=False)

display(
    al_df.head(), 
    cu_df.head(), 
    pb_df.head(), 
    ni_df.head(), 
    zn_df.tail()
)

,LME Aluminium Cash-Settlement,LME Aluminium 3-month,LME Aluminium stock
date,,,
2023-09-11,2154.0,2195.0,495.875
2023-09-08,2132.5,2181.0,499.475
2023-09-07,2135.0,2180.0,501.475
2023-09-06,2158.0,2206.0,502.4
2023-09-05,2136.5,2188.5,506.875


,LME Copper Cash-Settlement,LME Copper 3-month,LME Copper stock
date,,,
2023-09-11,8332.5,8360.0,134.525
2023-09-08,8225.0,8242.0,134.125
2023-09-07,8242.5,8270.0,133.85
2023-09-06,8395.0,8412.0,110.4
2023-09-05,8430.0,8430.0,107.4


,LME Lead Cash-Settlement,LME Lead 3-month,LME Lead stock
date,,,
2023-09-11,2314.0,2245.0,52.55
2023-09-08,2285.0,2219.0,52.975
2023-09-07,2287.0,2232.5,54.275
2023-09-06,2300.0,2231.0,55.65
2023-09-05,2274.0,2220.0,55.35


,LME Nickel Cash-Settlement,LME Nickel 3-month,LME Nickel stock
date,,,
2023-09-11,20050.0,20260.0,38.28
2023-09-08,19905.0,20125.0,37.98
2023-09-07,20130.0,20310.0,37.386
2023-09-06,20550.0,20720.0,37.416
2023-09-05,20800.0,20960.0,37.164


,LME Zinc Cash-Settlement,LME Zinc 3-month,LME Zinc stock
date,,,
2020-01-08,2406.0,2386.0,50.175
2020-01-07,2346.0,2337.0,50.5
2020-01-06,2335.0,2320.0,50.775
2020-01-03,2284.0,2271.0,51.125
2020-01-02,2299.0,2290.5,51.2
